## Библиотека Bokeh
Библиотека для создания интерактивной визуализации

In [1]:
%pip install bokeh

                                              0.0/6.8 MB ? eta -:--:--
                                              0.0/6.8 MB ? eta -:--:--
                                              0.0/6.8 MB ? eta -:--:--
                                              0.0/6.8 MB 186.2 kB/s eta 0:00:37
                                              0.0/6.8 MB 186.2 kB/s eta 0:00:37
                                              0.0/6.8 MB 186.2 kB/s eta 0:00:37
                                              0.0/6.8 MB 186.2 kB/s eta 0:00:37
                                              0.0/6.8 MB 186.2 kB/s eta 0:00:37
                                              0.0/6.8 MB 186.2 kB/s eta 0:00:37
                                              0.0/6.8 MB 89.3 kB/s eta 0:01:16
                                              0.1/6.8 MB 125.8 kB/s eta 0:00:54
                                              0.1/6.8 MB 125.8 kB/s eta 0:00:54
                                              0.1/6.8 MB 125.8 kB/s 


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np 
import pandas as pd 

In [3]:
from bokeh.io import  output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.palettes import Viridis
from bokeh.transform import log_cmap

In [4]:
ykt_df = pd.read_excel("data/cities_of_Yakutia.xlsx")
ykt_df

,город,население,площадь,широта,долгота
0,Якутск,318768,122,62.00,129.70
1,Нерюнгри,57009,99,56.66,124.72
2,Вилюйск,11095,15,63.74,121.65
3,Ленск,23479,49,60.72,114.94
4,Алдан,20595,33,58.61,125.40
5,Олёкминск,9213,12,60.38,120.44
6,Среднеколымск,3489,20,67.46,153.70
7,Покровск,9256,11,61.48,129.14
8,Мирный,35223,15,62.54,113.98
9,Нюрба,9786,19,63.29,118.36


In [5]:
# функция для преобразования сферической системы координат в цилиндрическую систему координат для проекции Меркатора

def to_mercator(lat, lon):
    R_EARTH = 6378137.0  

    x = R_EARTH * np.radians(lon)

    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + np.radians(lat)/2.0)) * (x/lon)

    return pd.DataFrame({"merc_x": x, "merc_y": y})

In [6]:
ykt_df.columns

Index(['город', 'население', 'площадь', 'широта', 'долгота'], dtype='object')

In [7]:
merc = to_mercator(ykt_df['широта'], ykt_df['долгота'])
merc

,merc_x,merc_y
0,1.443814e+07,8.859143e+06
1,1.388377e+07,7.690941e+06
2,1.354202e+07,9.284045e+06
3,1.279506e+07,8.561813e+06
4,1.395946e+07,8.096566e+06
5,1.340732e+07,8.484831e+06
6,1.710981e+07,1.028837e+07
7,1.437580e+07,8.736881e+06
8,1.268820e+07,8.988336e+06
9,1.317577e+07,9.171715e+06


In [8]:
# добвление в ykt_df датафрейма из merc
ykt_df[["merc_x", "merc_y"]] = merc
ykt_df

,город,население,площадь,широта,долгота,merc_x,merc_y
0,Якутск,318768,122,62.00,129.70,1.443814e+07,8.859143e+06
1,Нерюнгри,57009,99,56.66,124.72,1.388377e+07,7.690941e+06
2,Вилюйск,11095,15,63.74,121.65,1.354202e+07,9.284045e+06
3,Ленск,23479,49,60.72,114.94,1.279506e+07,8.561813e+06
4,Алдан,20595,33,58.61,125.40,1.395946e+07,8.096566e+06
5,Олёкминск,9213,12,60.38,120.44,1.340732e+07,8.484831e+06
6,Среднеколымск,3489,20,67.46,153.70,1.710981e+07,1.028837e+07
7,Покровск,9256,11,61.48,129.14,1.437580e+07,8.736881e+06
8,Мирный,35223,15,62.54,113.98,1.268820e+07,8.988336e+06
9,Нюрба,9786,19,63.29,118.36,1.317577e+07,9.171715e+06


## Визуализация

In [13]:
# показ в блоктоне
output_notebook()

Loading BokehJS ...

In [22]:
# источник данных
src = ColumnDataSource(ykt_df)

# цветовой градиент
cm = log_cmap(
    "население", 
    low=ykt_df["население"].min(), 
    high=ykt_df["население"].max(), 
    palette=Viridis[256])

# всплывающая информация
info = [
    ("Город", "@{город}"),
    ("Площадь", "@{площадь} кв.км"),
    ("Население", "@{население} чел")
]

# объект рисунка
fig = figure(x_axis_type="mercator", y_axis_type="mercator", tooltips=info)

# добавление точек
fig.circle(source=src, x="merc_x", y="merc_y", color=cm, size=10)

# добавление тайла
fig.add_tile("OSM")

# рендер
show(fig)